In [2]:
import plotly.graph_objects as go
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
from pymongo import MongoClient, DESCENDING, ASCENDING
import sys, os
import time
import pymongo
import plotly.graph_objects as go
import streamlit as st
import plotly.subplots as sp
# Get the current working directory
current_dir = os.getcwd()

# Construct the path to the directory containing the module
module_dir = os.path.abspath(os.path.join(current_dir, "../src/data/utils"))

# Add the directory to s ys.path
sys.path.append(module_dir)
from trading_strategy import TradingStrategy

### Main Page

In [ ]:
# MongoDB Configuration
DB_NAME = st.secrets['db_name']
WAREHOUSE_INTERVAL = st.secrets.warehouse_interval
WAREHOUSE_INTERVAL_COLLECTION = '1d_data'
PROCESSED_COLLECTION = st.secrets.processed_collection_name
ALERT_COLLECTION = st.secrets.alert_collection_name

def initialize_mongo_client():
    client = MongoClient(**st.secrets["mongo"])
    return client

@st.cache_data
def fetch_index_return(symbol):
    collection = initialize_mongo_client()[DB_NAME][WAREHOUSE_INTERVAL_COLLECTION]
    if not WAREHOUSE_INTERVAL:
        raise ValueError("warehouse_interval is empty in st.secrets")

    # Fetch data from MongoDB for the specified symbol and date range
    df = pd.DataFrame(list(collection.find(
        {"symbol": symbol, "date": {"$gte": pd.to_datetime("2024-01-01T00:00:00Z")}}, 
        {"_id": 0}
    )))

    # Ensure the DataFrame is sorted by date
    df = df.sort_values(by="date")

    # Calculate cumulative returns based on the first available 'close' value
    df['cumulative_return'] = (df['close'] / df['close'].iloc[0]) - 1

    return df

def fetch_portfolio_return(trades_data):
    trades_data = pd.DataFrame(trades_data)
    
def get_trade_data(data_collection, alert_collection):
    
    trades_history = TradingStrategy(data_collection, alert_collection)
    trades_history.execute_trades()
    return trades_history.get_trades()

@st.cache_data
def compute_metrics(filtered_trades):
    win_trades = len(filtered_trades[filtered_trades['profit'] > 0])
    loss_trades = len(filtered_trades[filtered_trades['profit'] <= 0])
    final_trade_profit_rate = (round(filtered_trades['total_asset'].iloc[-1]) - 10000) / 100
    return [win_trades, loss_trades, final_trade_profit_rate]
    
st.set_page_config(page_title="Stock Prediction Dashboard", layout="wide")
st.sidebar.success("Select a page to view")

client =  MongoClient(**st.secrets["mongo"])
symbols = client[st.secrets['db_name']][st.secrets.processed_collection_name].distinct("symbol")
st.title("Stock Prediction Dashboard")
st.header("Earning Line Chart YTD")

# Initialize the figure for the line chart
line_chart = go.Figure()

# Loop through the symbols (e.g., 'QQQ', 'SPY') to fetch index returns and add them to the plot
for symbol in ['QQQ', 'SPY']:
    data = fetch_index_return(symbol)[['cumulative_return', 'date']]
    line_chart.add_trace(go.Scatter(
        x=data['date'], 
        y=data['cumulative_return'],  # Ensure you use 'cumulative_return' as the y-axis
        mode='lines+markers', 
        name=symbol,
        marker_line_color="rgba(0,0,0,0.7)",
        opacity=0.7
    ))

# Initialize MongoDB client and fetch the necessary data
client = initialize_mongo_client()
symbols = client[st.secrets['db_name']][st.secrets['processed_collection_name']].distinct("symbol")

# Fetch trade data
df_trades = get_trade_data(client[DB_NAME][PROCESSED_COLLECTION], 
                        client[DB_NAME][ALERT_COLLECTION])

# Compute metrics (e.g., win rate, loss rate, final profit rate)
win_trades, loss_trades, final_trade_profit_rate = compute_metrics(df_trades)

# Create columns in Streamlit for display (if using Streamlit)
col1, col2, col3 = st.columns(3, gap='medium')

# Ensure 'Exit_date' is in datetime format
df_trades['Exit_date'] = pd.to_datetime(df_trades['Exit_date'])

# Compute cumulative return of trades
df_trades['cumulative_return'] = df_trades['total_asset'].pct_change().fillna(0).add(1).cumprod()

# Add the cumulative return of the trades to the line chart
line_chart.add_trace(go.Scatter(
    x=df_trades['Exit_date'],
    y=df_trades['cumulative_return'],
    mode='lines+markers',
    name='Portfolio',
    marker_line_color="rgba(0,0,0,0.7)",
    opacity=0.7
))

# CSS styling for metric containers
st.markdown("""
    <style>
    .metric-container {
        background-color: #f5f5f5;
        border-radius: 8px;
        padding: 20px;
        box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.1);
        display: flex;
        justify-content: center;
        align-items: center;
        flex-direction: column;
        text-align: center;
        height: 100%;
    }
    .metric-label {
        font-weight: bold;
        font-size: 24px;
        margin: 0;
    }
    .metric-value {
        font-size: 24px;
        font-weight: bold;
        margin-top: 8px;
    }
    </style>
""", unsafe_allow_html=True)

# Determine the color of the profit value based on its sign
profit_color = "green" if final_trade_profit_rate > 0 else "red"

with col1:
    st.markdown(f"""
        <div class="metric-container">
            <h3 class="metric-label">Final Trade Profit</h3>
            <p class="metric-value" style="color: {profit_color};">{final_trade_profit_rate}%</p>
        </div>
    """, unsafe_allow_html=True)
        
with col2:
    st.markdown(f"""
        <div class="metric-container">
            <h3 class="metric-label">Win Trades</h3>
            <p class="metric-value" style="color: green;">{win_trades}</p>
        </div>
    """, unsafe_allow_html=True)
        
with col3:
    st.markdown(f"""
        <div class="metric-container">
            <h3 class="metric-label">Loss Trades</h3>
            <p class="metric-value" style="color: red;">{loss_trades}</p>
        </div>
    """, unsafe_allow_html=True)

st.plotly_chart(line_chart)

col1, col2, col3 = st.columns(3)

with col1:
    query = {
        "date": {"$gte": pd.to_datetime("2024-10-11T00:00:00Z")},
        "interval": {"$in": [1,3,6,13]},
        "$expr": {
            "$and": [
                {"$gt": ["$close", "$13EMA"]},
                {"$gt": ["$close", "$169EMA"]}
            ]
        }
    }
    results = initialize_mongo_client()[DB_NAME][PROCESSED_COLLECTION].distinct("symbol", query)

    st.subheader("Buy Signal")
    if not results:
        st.write("No results found")
    else:
        # Add Buy specific styles
        st.markdown("""
            <style>
                .buy-container {
                    display: flex;
                    flex-wrap: wrap;
                    gap: 10px;
                }
                .buy-badge {
                    background-color: #4CAF50 !important;
                    color: white;
                    padding: 8px 12px;
                    border-radius: 5px;
                    font-size: 16px;
                    text-align: center;
                }
            </style>
        """, unsafe_allow_html=True)

        st.markdown('<div class="buy-container">', unsafe_allow_html=True)
        for symbol in results:
            st.markdown(f'<div class="buy-badge">{symbol}</div>', unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

with col2:
    query = {
        "date": {"$gte": pd.to_datetime("2024-10-11T00:00:00Z")},
        "interval": {"$in": [1,3,6,13]},
        "$expr": {
            "$and": [
                {"$lt": ["$close", "$13EMA"]},
                {"$gt": ["$close", "$169EMA"]}
            ]
        }
    }
    results = initialize_mongo_client()[DB_NAME][PROCESSED_COLLECTION].distinct("symbol", query)

    st.subheader("Hold Signal")
    if not results:
        st.write("No results found")
    else:
        # Add Hold specific styles
        st.markdown("""
            <style>
                .hold-container {
                    display: flex;
                    flex-wrap: wrap;
                    gap: 10px;
                }
                .hold-badge {
                    background-color: #A9A9A9 !important;
                    color: white;
                    padding: 8px 12px;
                    border-radius: 5px;
                    font-size: 16px;
                    text-align: center;
                }
            </style>
        """, unsafe_allow_html=True)

        st.markdown('<div class="hold-container">', unsafe_allow_html=True)
        for symbol in results:
            st.markdown(f'<div class="hold-badge">{symbol}</div>', unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

with col3:
    query = {
        "date": {"$gte": pd.to_datetime("2024-10-11T00:00:00Z")},
        "interval": {"$in": [1,3]},
        "$expr": {
            "$or": [
                {"$lt": ["$close", "$13EMA"]},
                {"$lt": ["$close", "$169EMA"]}
            ]
        }
    }
    results = initialize_mongo_client()[DB_NAME][PROCESSED_COLLECTION].distinct("symbol", query)

    st.subheader("Sell Signal")
    if not results:
        st.write("No results found")
    else:
        # Add Sell specific styles
        st.markdown("""
            <style>
                .sell-container {
                    display: flex;
                    flex-wrap: wrap;
                    gap: 10px;
                }
                .sell-badge {
                    background-color: #FF0000 !important;
                    color: white;
                    padding: 8px 12px;
                    border-radius: 5px;
                    font-size: 16px;
                    text-align: center;
                }
            </style>
        """, unsafe_allow_html=True)

        st.markdown('<div class="sell-container">', unsafe_allow_html=True)
        for symbol in results:
            st.markdown(f'<div class="sell-badge">{symbol}</div>', unsafe_allow_html=True)
        st.markdown('</div>', unsafe_allow_html=True)

### Intrade Day Dashbaord

In [ ]:
# MongoDB Configuration
DB_NAME = st.secrets['db_name']
PROCESSED_COLLECTION_NAME = st.secrets.processed_collection_name
STREAMING_COLLECTIONS = [f'{interval}_stock_datastream' for interval in st.secrets.streaming_interval]
LIVE_ALERT_COLLECTION = st.secrets.live
BATCH_ALERT_COLLECTION = st.secrets.batch
DESIRED_STREAMING_INTERVAL = st.secrets.streaming_interval
WINDOW_SIZE = st.secrets.window_size

def initialize_mongo_client(db_name=DB_NAME):
    client = pymongo.MongoClient(**st.secrets["mongo"])
    return client[db_name]

@st.cache_data
def get_current_date():
    current_date = pd.to_datetime('today').normalize()
    if current_date.weekday() in [5, 6]:  # Saturday or Sunday
        current_date -= pd.Timedelta(days=current_date.weekday() - 4)
    elif current_date.weekday() == 0 and pd.to_datetime('today').hour < 9:  # Monday before 9:30
        current_date -= pd.Timedelta(days=3)
    elif pd.to_datetime('today').hour < 7:  # Weekday before 9:30
        current_date -= pd.Timedelta(days=1)
    return (current_date.replace(hour=7, minute=30) - pd.Timedelta(days=WINDOW_SIZE))

def fetch_data(db, collection_name, symbol):
    query = {"symbol": symbol}
    return db[collection_name].find(query)

def fetch_alerts(db, collection_name, symbol, interval, date, alert_type=None):
    query = {"symbol": symbol, "interval": interval, "datetime": {"$gte": date}}
    if alert_type:
        query["alert_type"] = {"$in": alert_type}
    
    # Fetch the latest document by sorting 'datetime' in descending order
    return db[collection_name].find(query, sort=[("datetime", -1)])

def plot_candlestick_chart(filtered_df,filtered_live_alerts, stock_selector, interval):
    candlestick_chart = go.Figure(data=[go.Candlestick(
        x=filtered_df['datetime'], 
        open=filtered_df['open'], 
        high=filtered_df['high'], 
        low=filtered_df['low'], 
        close=filtered_df['close']
    )])
    
    for _, alert in filtered_live_alerts.iterrows():
        color = 'green' if alert['alert_type'] in ['bullish_engulfer', 'bullish_382'] else 'red' if alert['alert_type'] == 'bearish_engulfer' else 'black'
        hover_text = f"Alert Type: {alert['alert_type']}<br>Date: {alert['datetime'].strftime('%Y-%m-%d %H:%M:%S')}"
        candlestick_chart.add_annotation(
            x=alert['datetime'],
            y=1.05,
            xref='x',
            yref='paper',
            showarrow=False,
            text='',
            hovertext=hover_text,
            font=dict(color=color),
            bgcolor=color,
            bordercolor=color,
            borderwidth=2
        )
    
    # for _, row in filtered_batch_alerts.iterrows():
    #     end_date = row['datetime'] + pd.Timedelta(minutes=60)
    #     row['datetime'] = row['datetime'].strftime('%Y-%m-%d %H:%M:%S')
    #     end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
    #     for line_type, color in [('support', 'blue'), ('resistance', 'red')]:
    #         if line_type in row:
    #             candlestick_chart.add_trace(go.Scatter(
    #                 x=[row['datetime'], end_date], 
    #                 y=[row[line_type], row[line_type]], 
    #                 mode='lines', 
    #                 line=dict(color=color, width=1, dash='dash'), 
    #                 name=line_type.capitalize()
    #             ))
    
    candlestick_chart.update_layout(
        xaxis_rangeslider_visible=False,
        showlegend=False,
        xaxis_type='category',
        title={
            'text': f'{stock_selector}',
            'x': 0.5,  # Center the title
            'xanchor': 'center',  # Anchor the title at the center
            'yanchor': 'top',  # Anchor the title at the top
            'font': {
                'size': 24,  # Set the font size
                'color': 'black',  # Set the font color
                'family': 'Arial, sans-serif'  # Set the font family
            }
        },
        xaxis=dict(
            showticklabels=False,
            title=''
        ),
        yaxis=dict(
            title='Price'
        ),
        width=1200,
        height=700
        )
    return candlestick_chart

def plot_support_resistance_histogram(filtered_batch_alerts):
    if 'support' in filtered_batch_alerts.columns or 'resistance' in filtered_batch_alerts.columns:
        histogram_fig = go.Figure()

        if 'support' in filtered_batch_alerts.columns:
            histogram_fig.add_trace(go.Histogram(
                x=filtered_batch_alerts['support'],
                name='Support',
                marker_color='blue',
                opacity=0.5,
                xbins=dict(size=0.5)
            ))

        if 'resistance' in filtered_batch_alerts.columns:
            histogram_fig.add_trace(go.Histogram(
                x=filtered_batch_alerts['resistance'],
                name='Resistance',
                marker_color='red',
                opacity=0.5,
                xbins=dict(size=0.5)
            ))

        histogram_fig.update_layout(
            barmode='overlay',
            xaxis_title='Price',
            yaxis_title='Count',
            showlegend=False,
            title={
            'text': 'Support/Resistance Levels Strength',
            'x': 0.5,  # Center the title
            'xanchor': 'center',  # Anchor the title at the center
            'yanchor': 'top',  # Anchor the title at the top
            'font': {
                'size': 18,  # Set the font size
                'color': 'black',  # Set the font color
                'family': 'Arial, sans-serif'  # Set the font family
            }
            },
            xaxis=dict(
                showticklabels=True,
                title='Price'
            ),
            yaxis=dict(
                title='Strength'
            )
        )
        st.plotly_chart(histogram_fig, use_container_width=True,showlegend=False)

    else:
        st.warning("No support/resistance data available for the selected interval and symbol.")
            
def plot_live_page(db, stock_selector, interval):
    # Prepare data for the selected stock and interval
    selected_topic = f"{interval}_stock_datastream"
    filtered_query = fetch_data(db, selected_topic, stock_selector)
    filtered_live_alerts_query = fetch_alerts(db, LIVE_ALERT_COLLECTION, stock_selector, interval, date=get_current_date(),
                                            alert_type=["bullish_engulfer", "bearish_engulfer", "bullish_382"])
    filtered_batch_alerts_query = fetch_alerts(db, BATCH_ALERT_COLLECTION, stock_selector, interval,date=get_current_date())
    
    # Convert the query results to a DataFrame
    filtered_df = pd.DataFrame(list(filtered_query))
    filtered_live_alerts = pd.DataFrame(list(filtered_live_alerts_query))
    filtered_batch_alerts = pd.DataFrame(list(filtered_batch_alerts_query))
    # Convert the 'datetime' field to datetime
    if not filtered_df.empty and not filtered_live_alerts.empty:
        # Ensure consistency in the datetime format
        filtered_df['datetime'] = pd.to_datetime(filtered_df['datetime'])
        filtered_live_alerts['datetime'] = pd.to_datetime(filtered_live_alerts['datetime'])
        
    else:
        st.warning("No data available for the selected interval and symbol.")
        return
    # Plot the Chart
    candlestick_chart = plot_candlestick_chart(filtered_df, filtered_live_alerts, stock_selector, interval)
    # Display the chart
    st.plotly_chart(candlestick_chart, use_container_width=True)
    
    # Display the lastest alerts
    if not filtered_live_alerts.empty:
        col1, col2 = st.columns(2)
        
        with col1:
            
            only_time = filtered_live_alerts['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
            only_time = only_time.head(1).values[0]
                
            st.markdown(f"""
            <div style="
                text-align: center; 
                font-size: 36px; 
                font-weight: bold; 
                color: #4CAF50; 
                border: 2px solid #4CAF50; 
                padding: 10px; 
                border-radius: 10px;
                background-color: #f9f9f9;
            ">
            {only_time}
            </div>
            """, unsafe_allow_html=True)
            
        with col2: 
            alert_type = filtered_live_alerts['alert_type'].values[0]
            
            # Determine the color and message based on the alert type
            if alert_type == 'bullish_engulfer' or alert_type == 'bullish_382':
                alert_message = f"{alert_type.replace('_', ' ').capitalize()} &#x1F7E2;"  # Green Circle Emoji
                alert_color = "#4CAF50"  # Green color
            elif alert_type == 'bearish_engulfer':
                alert_message = f"{alert_type.replace('_', ' ').capitalize()} &#x1F534;"  # Red Circle Emoji
                alert_color = "#FF5733"  # Red color
            st.markdown(f"""
            <div style="
                text-align: center; 
                font-size: 36px; 
                font-weight: bold; 
                color: {alert_color}; 
                border: 2px solid {alert_color}; 
                padding: 10px; 
                border-radius: 10px;
                background-color: #f9f9f9;
            ">
            {alert_message}
            </div>
            """, unsafe_allow_html=True)
    else:      
        col1, col2 = st.columns(2)
        with col1:
                    st.markdown(f"""
            <div style="
                text-align: center; 
                font-size: 36px; 
                font-weight: bold; 
                color: #808080; 
                border: 2px solid #808080; 
                padding: 10px; 
                border-radius: 10px;
                background-color: #f9f9f9;
            ">
            No Alert Time Available
            </div>
            """, unsafe_allow_html=True)
            
        with col2: 
            st.markdown(f"""
            <div style="
                text-align: center; 
                font-size: 36px; 
                font-weight: bold; 
                color: #808080;
                border: 2px solid #808080; 
                padding: 10px; 
                border-radius: 10px;
                background-color: #f9f9f9;
            ">
            No Alert Yet
            </div>
            """, unsafe_allow_html=True)
            
    # Display the support/resistance histogram
    plot_support_resistance_histogram(filtered_batch_alerts)
    
            
if __name__ == "__main__":
    # Set page config
    st.set_page_config(layout="wide")
    # Initialize MongoDB client
    db = initialize_mongo_client()
    # Get the list of stocks
    options = sorted(db[STREAMING_COLLECTIONS[0]].distinct("symbol"))
    intervals = DESIRED_STREAMING_INTERVAL
    # Streamlit UI
    st.sidebar.header("Selector")
    st.title(f'Live Alert Tracking Dashboard')
    stock_selector = st.sidebar.selectbox('Select Stock', options=options, index=0)
    intervals_selector = st.sidebar.selectbox('Select Interval', options=intervals, index=1)
    # Create a placeholder for the chart
    chart_placeholder = st.empty()
    # Continuously update the chart every minute
    while True:
        with chart_placeholder.container():
            plot_live_page(db, stock_selector, intervals_selector)
        # Refresh the data every 60 seconds
        time.sleep(60)

### Long Term Stock Dashboard

In [ ]:
# MongoDB Configuration
DB_NAME = st.secrets['db_name']
PROCESSED_COLLECTION_NAME = st.secrets.processed_collection_name
ALERT_COLLECTION_NAME = st.secrets.alert_collection_name
def connect_to_mongo(db_name=DB_NAME):
    client = pymongo.MongoClient(**st.secrets["mongo"])
    return client[db_name]


def fetch_stock_data(collection, stock_symbol, interval):
    warehouse_interval = st.secrets.warehouse_interval
    if not warehouse_interval:
        raise ValueError("warehouse_interval is empty in st.secrets")
    return pd.DataFrame(list(collection.find({"symbol": stock_symbol,
                                            "interval": interval}, 
                                            {"_id": 0}))).sort_values(by=['date'])

def fetch_alert_data(collection, stock_symbol,interval):
    if not interval:
        raise ValueError("warehouse_interval is empty in st.secrets")
    query = collection.find({"symbol": stock_symbol,
                            "interval": interval},
                            {"_id": 0})

    return pd.DataFrame(list(query)).sort_values(by=['date'])     

def create_figure(filtered_df, show_macd=False):

    row_height = [0.8, 0.2] if show_macd else [1]
    row = 2 if show_macd else 1
    # Calculate the date range for the last 6 months
    end_date = filtered_df['date'].max() + pd.DateOffset(days=20)
    start_date = end_date - pd.DateOffset(months=6)
    
    # Calculate the price range for the last 6 months
    price_range = filtered_df[(filtered_df['date'] >= start_date) & (filtered_df['date'] <= end_date)]
    min_price = price_range['low'].min() * 0.95 
    max_price = price_range['high'].max() * 1.05
    
    fig = sp.make_subplots(rows=row, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=row_height)

    fig.add_trace(go.Candlestick(
        x=filtered_df['date'],
        open=filtered_df['open'],
        high=filtered_df['high'],
        low=filtered_df['low'],
        close=filtered_df['close'],
        name='price'), row=1, col=1)
    fig.update_xaxes(range=[start_date, end_date],title_text="Date", row=3, col=1)
    
    for ema in ['144EMA', '169EMA', '13EMA', '8EMA']:
        fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df[ema], mode="lines", name=ema), row=1, col=1)
        
    fig.update_xaxes(range=[start_date, end_date],title_text="Date", row=1, col=1)
    fig.update_yaxes(range=[min_price, max_price], title_text="Price", row=1, col=1)
    
    # for trade_type, color in [('Entry', 'rgba(0,255,0,0.7)'), ('Exit', 'rgba(255,0,0,0.7)')]:
    #     fig.add_trace(go.Scatter(
    #         x=filtered_trades[f'{trade_type}_date'],
    #         y=filtered_trades[f'{trade_type}_price'],
    #         mode="markers",
    #         customdata=filtered_trades,
    #         marker_symbol="diamond-dot",
    #         marker_size=8,
    #         marker_line_width=2,
    #         marker_line_color="rgba(0,0,0,0.7)",
    #         marker_color=color,
    #         hovertemplate=f"{trade_type} Time: %{{customdata[1]}}<br>{trade_type} Price: %{{y:.2f}}<br>Total Asset: %{{customdata[5]:.3f}}",
    #         name=f"{trade_type}s"), row=1, col=1)

    if show_macd:
        fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['MACD'], mode='lines', name='MACD'), row=2, col=1)
        fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['MACD_SIGNAL'], mode='lines', name='MACD signal'), row=2, col=1)
        
    # Calculate the MACD start and end date
    macd_start_date = filtered_df['date'].min()
    macd_end_date = filtered_df['date'].max()
    
    fig.update_xaxes(range=[start_date, end_date],title_text="Date", row=2, col=1)
    fig.update_yaxes(range=[macd_start_date, macd_end_date], title_text="MACD", row=2, col=1)
    # fig.add_trace(go.Scatter(x=filtered_trades['Exit_date'], y=filtered_trades['total_asset'], mode='lines', name='Profit', line=dict(color='green')), 
    #             row=row, col=1)\
    # fig.update_yaxes(title_text="Profit/Loss", row=3, col=1)
    
    fig.update_layout(xaxis_rangeslider_visible=False, autosize=False, showlegend=False, height=1000, width=1200)

    return fig

def display_alerts(alert_df):
    # Display Alert
    today_alert = alert_df[alert_df['date'] == alert_df['date'].max()]

    # Function to map alert values to color and message
    def get_alert_color_and_message(alert_type, value):
        alert_mappings = {
            'velocity_alert': {
                'velocity_maintained': ('green', 'Velocity Maintained'),
                'velocity_weak': ('red', 'Velocity Weakened'),
                'velocity_loss': ('red', 'Velocity Loss'),
                'velocity_negotiating': ('red', 'Velocity Negotiating')
            },
            'candle_alert': {
                'bullish_engulf': ('green', 'Bullish Engulfing'),
                'bearish_engulf': ('red', 'Bearish Engulfing'),
                'hammer': ('green', 'Hammer Pattern'),
                'inverse_hammer': ('red', 'Inverse Hammer Pattern')
            },
            'macd_alert': {
                'bullish_macd': ('green', 'MACD Bullish Crossover'),
                'bearish_macd': ('red', 'MACD Bearish Crossover')
            }
        }
    
        # Default to grey color if value or alert type is not recognized
        color, message = alert_mappings.get(alert_type, {}).get(value, ('grey', 'Unknown Alert'))
        return color, message
    
    # Main function to display alerts
    def display_alerts(today_alert):        
        # Loop through the relevant alert columns (e.g., velocity, candle, MACD)
        for column in ['velocity_alert', 'candle_alert', 'macd_alert']:
            alert_value = today_alert[column].values[0] if not today_alert[column].empty else None
            if pd.notna(alert_value):  # Only display if alert has a valid value
                alert_color, alert_message = get_alert_color_and_message(column, alert_value)
            
                # Display the alert with color-coded dot and message
                st.markdown(f"""
                    <div style="text-align: center;">
                    <span style="font-size:50px; color:{alert_color}">●</span>
                    <div style="font-size:16px; font-weight:bold; margin-top:10px; color:{alert_color};">{alert_message}</div>
                    </div>
                """, unsafe_allow_html=True)
                
    # Display the alerts            
    display_alerts(today_alert)
    
def static_analysis_page(processed_collection, alert_collection):
    # Set the page title and layout
    st.set_page_config(page_title="Stock Analysis Dashboard", layout="wide")
    st.markdown("<h1 style='text-align: center;'>Stock Analysis Dashboard</h1>", unsafe_allow_html=True)
    
    # Add a sidebar
    # Create a dropdown to select the stock
    stock_selector = st.sidebar.selectbox('Select Stock', options=sorted(processed_collection.distinct("symbol")), index=0)
    # Create a dropdown to select the interval
    default_interval = st.secrets['velocity_dict'][stock_selector]
    interval_selector = st.sidebar.selectbox('Optimal Interval/ Select Interval', 
                                    options=sorted(processed_collection.distinct("interval")), 
                                    index=sorted(processed_collection.distinct("interval")).\
                                        index(default_interval) if default_interval in processed_collection.distinct("interval")\
                                            else 0)
    
    alert_df = fetch_alert_data(alert_collection, stock_selector, interval_selector)
    # Add an update button
    if st.sidebar.button("Update Data"):
        # Refetch the latest data when the button is clicked
        processed_df = fetch_stock_data(processed_collection, stock_selector, interval_selector)
        st.success("Data updated successfully!")
    else:
        # Display the existing data if the button is not clicked
        processed_df = fetch_stock_data(processed_collection, stock_selector, interval_selector)
        
    # Add a checkbox to show the MACD
    show_macd = st.sidebar.checkbox("Show MACD", value=False)
    
    # Create the figure
    fig = create_figure(processed_df, show_macd)
    
    col1, col2 = st.columns([5, 1], vertical_alignment="top")
    with col1:
        st.plotly_chart(fig, use_container_width=True)
    
    with col2:
        # Display the alerts
        display_alerts(alert_df)
    
if __name__ == "__main__":
    # Connect to MongoDB and fetch the processed collection
    processed_collection = connect_to_mongo()[PROCESSED_COLLECTION_NAME]
    alert_collection = connect_to_mongo()[ALERT_COLLECTION_NAME]
    static_analysis_page(processed_collection, alert_collection)